In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
import pickle    

In [ ]:
data= pd.read_csv('Churn_Modelling.csv')


In [ ]:
data = data.drop(columns=["RowNumber", "CustomerId", "Surname"])

In [ ]:
#  encode categorical variables
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data

In [ ]:
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

In [ ]:
geo_encoded_df = pd.DataFrame(
    geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out()
)

In [ ]:
# combile one hot encoded columns with the original data

data = pd.concat([data.drop(columns=["Geography"]), geo_encoded_df], axis=1)
data.head()

In [ ]:
#  divide the dataset into independent and dependent features
X = data.drop(columns=["EstimatedSalary"]
)  # these are the independent features to predict the target variable
y = data["EstimatedSalary"]  # this is the target variable

In [ ]:
#  split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:

# save the encoders and scaler

with open("label_encoder_gender.pkl", "wb") as f:
    pickle.dump(label_encoder_gender, f)
with open("onehot_encoder_geo.pkl", "wb") as f:
    pickle.dump(onehot_encoder_geo, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

## ANN Regression Problem Statement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  # used to create a sequential model
from tensorflow.keras.layers import Dense  # used to create a dense layer

In [ ]:
# build our ANN model
num_input_layers = (X_train.shape[1],)


model = Sequential(
    [
        Dense(
            64, activation="relu", input_shape=num_input_layers
        ),  # HL1 connected to input layer
        Dense(32, activation="relu"),  # HL2 connected to previous HL
        Dense(1),  # output layer for binary classification
    ]
)

In [ ]:
# compile the model
model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mae"])

In [ ]:
#  setup tensorboard 
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# setup up early stopping
early_stopping_callback = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

In [ ]:
# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback],
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regression_logs/fit

In [ ]:
#  evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, MAE: {mae}")

In [ ]:
#  save the model
model.save("salary_regression_model.h5")